In [1]:
import torch
import numpy as np

project_dir = os.path.split(os.getcwd())[0]
if project_dir not in sys.path:
    sys.path.append(project_dir)

from models.Paper import CNN
from InformationPlane import TensorKernel, MutualInformation

In [2]:
a = torch.rand((4,1,3,3), dtype=torch.float32)
b = torch.zeros((4,3), dtype=torch.float32)
b[0,1] = 1; b[1,2] = 1; b[2,0] = 1; b[3,0] = 1

print(a)
print(b)

tensor([[[[0.2284, 0.4067, 0.4654],
          [0.7116, 0.0048, 0.3853],
          [0.1416, 0.5270, 0.9954]]],


        [[[0.1680, 0.2708, 0.2652],
          [0.5976, 0.7449, 0.2521],
          [0.2363, 0.2844, 0.1719]]],


        [[[0.3211, 0.0410, 0.7893],
          [0.5093, 0.3024, 0.8547],
          [0.5465, 0.7314, 0.3550]]],


        [[[0.4949, 0.0583, 0.5554],
          [0.2544, 0.3589, 0.2529],
          [0.6843, 0.3039, 0.6006]]]])
tensor([[0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.]])


In [3]:
a_flatten = torch.flatten(a, 1)

# Tensor kernels definitions

def RBF(x, y, sigma):
    distance = torch.dist(x,y)
    return torch.exp(-(distance**2)/(sigma**2))

def RBF_2(x, sigma):
    distance = torch.cdist(x, x)
    return torch.exp(-(distance**2)/(sigma**2))

# Ambas funciones dan el mismo resultado, lo doy por bueno


In [4]:
rbf_a = RBF_2(a_flatten, 8)
rbf_b = RBF_2(b, 0.9)

print(rbf_a)
print(rbf_b)

tensor([[1.0000, 0.9786, 0.9812, 0.9837],
        [0.9786, 1.0000, 0.9807, 0.9862],
        [0.9812, 0.9807, 1.0000, 0.9879],
        [0.9837, 0.9862, 0.9879, 1.0000]])
tensor([[1.0000, 0.0847, 0.0847, 0.0847],
        [0.0847, 1.0000, 0.0847, 0.0847],
        [0.0847, 0.0847, 1.0000, 1.0000],
        [0.0847, 0.0847, 1.0000, 1.0000]])


In [5]:
k_0 = torch.zeros((len(a_flatten), len(a_flatten)))
for i in range(len(a_flatten)):
    for j in range(len(a_flatten)):
        k_0[i,j] = RBF(a_flatten[i], a_flatten[j], 8)
        
print(k_0)

tensor([[1.0000, 0.9786, 0.9812, 0.9837],
        [0.9786, 1.0000, 0.9807, 0.9862],
        [0.9812, 0.9807, 1.0000, 0.9879],
        [0.9837, 0.9862, 0.9879, 1.0000]])


In [6]:
k_1 = torch.zeros((len(b), len(b)))
for i in range(len(b)):
    for j in range(len(b)):
        k_1[i,j] = RBF(b[i], b[j], 0.1)
        
print(k_1)

tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 1.],
        [0., 0., 1., 1.]])


# Kernel Aligment Loss

In [7]:
def kernelAligmentLoss(x, y):
    return ((torch.sum(x*y))/(torch.norm(x) * torch.norm(y))).item()

In [8]:
kernelAligmentLoss(rbf_a, rbf_b)

0.6995184421539307

# Optimize sigma value

In [9]:
mean_distance = torch.tensor([torch.dist(a_flatten[i-1], a_flatten[i]) for i in range(1, len(a_flatten))]).mean()

distance = torch.cdist(a_flatten, a_flatten)
mean_distance_v2 = distance[distance != 0].mean()

print(a_flatten)
print(mean_distance)
print(mean_distance_v2)

possible_sigma_values = np.arange(mean_distance*0.1, mean_distance*10, (mean_distance*10 - mean_distance*0.1)/75)
possible_sigma_values_v2 = np.arange(mean_distance_v2*0.1, mean_distance_v2*10, (mean_distance_v2*10 - mean_distance_v2*0.1)/75)

print(possible_sigma_values)
print(possible_sigma_values_v2)

# print(possible_sigma_values_v2.tolist())

test = list(map(lambda sigma: RBF_2(a_flatten, sigma), possible_sigma_values_v2.tolist()))
print(test[70])

label_kernel = rbf_b
result = np.array(list(map(lambda x: kernelAligmentLoss(x, label_kernel), test)))
print(result)

where = np.argwhere(result == result.max()).item()
print(where)

best_sigma_value = possible_sigma_values_v2[where]
print(best_sigma_value)

tensor([[0.2284, 0.4067, 0.4654, 0.7116, 0.0048, 0.3853, 0.1416, 0.5270, 0.9954],
        [0.1680, 0.2708, 0.2652, 0.5976, 0.7449, 0.2521, 0.2363, 0.2844, 0.1719],
        [0.3211, 0.0410, 0.7893, 0.5093, 0.3024, 0.8547, 0.5465, 0.7314, 0.3550],
        [0.4949, 0.0583, 0.5554, 0.2544, 0.3589, 0.2529, 0.6843, 0.3039, 0.6006]])
tensor(1.0587)
tensor(1.0409)
[ 0.10587024  0.24561894  0.38536764  0.52511634  0.66486504  0.80461374
  0.94436244  1.08411114  1.22385984  1.36360854  1.50335724  1.64310594
  1.78285464  1.92260334  2.06235204  2.20210074  2.34184944  2.48159814
  2.62134684  2.76109554  2.90084424  3.04059294  3.18034164  3.32009034
  3.45983904  3.59958774  3.73933644  3.87908514  4.01883384  4.15858254
  4.29833124  4.43807994  4.57782864  4.71757734  4.85732604  4.99707474
  5.13682344  5.27657214  5.41632084  5.55606954  5.69581824  5.83556694
  5.97531564  6.11506434  6.25481304  6.39456174  6.53431044  6.67405914
  6.81380784  6.95355654  7.09330524  7.23305394  7.37280

In [10]:
rbf_c = TensorKernel.RBF(a_flatten, 8)
I = MutualInformation(rbf_c, rbf_b)
print(rbf_c)
rbf_c[0, 0] = 0
print(rbf_c)
print(I.input_kernel)

tensor([[1.0000, 0.9786, 0.9812, 0.9837],
        [0.9786, 1.0000, 0.9807, 0.9862],
        [0.9812, 0.9807, 1.0000, 0.9879],
        [0.9837, 0.9862, 0.9879, 1.0000]])
tensor([[0.0000, 0.9786, 0.9812, 0.9837],
        [0.9786, 1.0000, 0.9807, 0.9862],
        [0.9812, 0.9807, 1.0000, 0.9879],
        [0.9837, 0.9862, 0.9879, 1.0000]])
tensor([[0.0000, 0.9786, 0.9812, 0.9837],
        [0.9786, 1.0000, 0.9807, 0.9862],
        [0.9812, 0.9807, 1.0000, 0.9879],
        [0.9837, 0.9862, 0.9879, 1.0000]])


In [11]:
print(I.optimizeSigmaValue(a_flatten))

NameError: name 'np' is not defined